In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools as it
from scipy.sparse import coo_matrix
%matplotlib inline

from lsq_code import remove_outlier, create_vandermonde, solve_linear_LS, solve_linear_LS_gd, mnist_pairwise_LS

# Other possibly useful functions
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score, confusion_matrix

### Exercise 1

When $n=1$, we can fit a degree-$m$ polynomial by choosing $f_{j}(x)=x^{j-1}$ and $M=m+1$. 
In this case, it follows that $A_{i,j}=x_{i}^{j-1}$
and the matrix $A$ is called a Vandermonde matrix.
Write a function to create Vandermonde matrix **(5 pt)**

In [ ]:
x = np.arange(1, 10)
create_vandermonde(x, 3)

### Exercise 2

Write a function to solve least-square problem via linear algebra **(5 pt)**

Implementation hint: check `numpy.linalg.lstsq`.

Using the setup in the previous example, try fitting the points $(1,2),(2,3),(3,5),(4,7),(5,11),(6,13)$
to a degree-2 polynomial.

Print the mean squared error. **(5 pt)**

Plot this polynomial (for $x\in[0,7]$) along with the data points to see the quality of fit. **(5 pt)**

In [ ]:
x = np.array([1, 2, 3, 4, 5, 6])
y = np.array([2, 3, 5, 7, 11, 14])
m = 2

# Create Vandermonde matrix A
A = 

# Use linear algebra to solve least-squares problem and minimize || y - A z ||^2
z_hat = 

# Compute the minimum square error
mse = 

# Generate x/y plot points for the fitted polynomial
xx = np.linspace(0, 7)
yy = 

plt.figure(figsize=(8, 4))
plt.scatter(x, y, color='red', label='data points')
plt.plot(xx, yy, linestyle='dotted',label='normal equation poly fit')
plt.legend()

poly1_expr = ' + '.join(['{0:.4f} x^{1}'.format(v, i) for i, v in enumerate(z_hat)][::-1])[:-4]
print('normal equation polynomial fit is {0}'.format(poly1_expr))
print('normal equation MSE is {0:.4f}'.format(mse))

### Exercise 3

Write a function to solve a least-squares problem via gradient descent. **(5 pt)**

Print the mean squared error. **(5 pt)**

Plot the resulting polynomial (for $x\in[0,7]$) along with previous polynomial and original data points to see the quality of fit. **(5 pt)**

In [ ]:
# Use gradient descent to solve least-squares problem and minimize || y - A z2 ||^2
z2_hat = 

# Compute the minimum square error
mse2 = 

# Generate y plot points for the gd fitted polynomial
yy2 =

plt.figure(figsize=(8, 4))
plt.scatter(x, y, color='red', label='data points')
plt.plot(xx, yy, linestyle='dotted',label='normal equation poly fit')
plt.plot(xx, yy2, linestyle='dashed', label='gradient descent poly fit')
plt.legend()

poly2_expr = ' + '.join(['{0:.4f} x^{1}'.format(v, i) for i, v in enumerate(z2_hat)][::-1])[:-4]
print('gradient descent polynomial fit is {0}'.format(poly2_expr))
print('gradient descent MSE is {0:.4f}'.format(mse2))

#### MNIST

Read `mnist_train.csv`, create a dataframe with two columns, column `feature` contains all $x$ and column `label` contains all $y$.

Plot the first 30 images.

In [ ]:
# read mnist csv file to a dataframe
df = pd.read_csv('mnist_train.csv')
# append feature column by merging all pixel columns
df['feature'] = df.apply(lambda row: row.values[1:], axis=1)
# only keep feature and label column
df = df[['feature', 'label']]
# display first 5 rows of the dataframe
df.head()

# Plot the first 30 images
plt.figure(figsize=(15, 2.5))
for i, row in df.iloc[:30].iterrows():
    x, y = row['feature'], row['label']
    plt.subplot(2, 15, i + 1)
    plt.imshow(x.reshape(28, 28), cmap='gray')
    plt.axis('off')
    plt.title(y)

### Exercise 4

Write the function `extract_and_split` to extract the all samples labeled with digit $n$ and randomly separate fraction of samples into training and testing groups. **(10 pt)**

Implementation hint: check `sklearn.model_selection.train_test_split`.

Pairwise experiment for applying least-square to classify digit $a$ and digit $b$. 

Follow the given steps in the template and implement the `mnist_pairwise_LS` function for pairwise experiment **(15 pt)**

Possible implementation hint: check `sklearn.metrics.accuracy_score`, `sklearn.metrics.confusion_matrix`

In [ ]:
# Pairwise experiment for LSQ to classify between 0 and 1
mnist_pairwise_LS(df, 0, 1, verbose=True)

### Exercise 5

Repeat the above problem for all pairs of digits. For each pair of digits, report the classification error rates for the training and testing sets. The error rates can be formatted nicely into a triangular matrix.  Put testing error in the lower triangle and training error in the upper triangle.

The code is given here in order demonstrate tqdm.
Points awarded for reasonable values **(10 pt)**

In [ ]:
from tqdm import tqdm_notebook as tqdm
num_trial, err_matrix = 1, np.zeros((10, 10))
for a, b in tqdm(it.combinations(range(10), 2), total=45):
    err_tr, err_te = np.mean([mnist_pairwise_LS(df, a, b) for _ in range(num_trial)], axis=0)
    err_matrix[a, b], err_matrix[b, a] = err_tr, err_te

print(np.round(err_matrix*100, 2))

### Exercise 6

But, what about a multi-class classifier for MNIST digits? 
For multi-class linear classification with d classes, one standard approach is to learn a linear mapping $f \colon \mathbb{R}^n \to \mathbb{R}^d $ where the “$y$”-value for the $i$-th class is chosen to be the standard basis vector $ \underline{e}_i \in \mathbb{R}^d $. 
This is sometimes called one-hot encoding. 
Using the same $A$ matrix as before and a matrix $Y$, defined by $Y_{i,j}$ if observation $i$ in class $j$ and $Y_{i,j} = 0$ otherwise, we can solve for the coefficient matrix $Z \in \mathbb{R}^d$ coefficients .
Then, the classifier maps a vector $\underline{x}$ to class $i$ if the $i$-th element of $Z^T \underline{x}$ is the largest element in the vector. 

Follow the steps in the template and implement the multi-class classification experiment **(20 pt)**

In [ ]:
# Randomly split into training/testing set
tr, te = 

# Construct the training set
X_tr = 
y_tr =

# Construct the testing set
X_te = 
y_te =

# Apply one-hot encoding to training labels
Y = 

# Run least-square on training set
Z =

# Compute estimation and misclassification on training set
y_hat_tr = 
err_tr =

# Compute estimation and misclassification on training set
y_hat_te =
err_te =

print('training error = {0:.2f}%, testing error = {1:.2f}%'.format(100 * err_tr, 100 * err_te))
# Compute confusion matrix
cm = np.zeros((10, 10), dtype=np.int64)
for a in range(10):
    for b in range(10):
        cm[a, b] = ((y_te == a) & (y_hat_te == b)).sum()
print('Confusion matrix:\n {0}'.format(cm))


